In [1]:
import pandas as pd
pd.options.display.max_columns = None
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import ranksums

In [2]:
#reading the dataset.
data = pd.read_csv('student-mat.csv', sep = ';')
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


In [3]:
data.shape

(395, 33)

In [4]:
#creating the colum wich will say if the student was passed or not.
data['Passed'] = data['G3'] >= 10
data.head(8)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,Passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6,False
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6,False
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10,True
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15,True
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10,True
5,GP,M,16,U,LE3,T,4,3,services,other,reputation,mother,1,2,0,no,yes,yes,yes,yes,yes,yes,no,5,4,2,1,2,5,10,15,15,15,True
6,GP,M,16,U,LE3,T,2,2,other,other,home,mother,1,2,0,no,no,no,no,yes,yes,yes,no,4,4,4,1,1,3,0,12,12,11,True
7,GP,F,17,U,GT3,A,4,4,other,teacher,home,mother,2,2,0,yes,yes,no,no,yes,yes,no,no,4,1,4,1,1,1,6,6,5,6,False


In [5]:
encoder = LabelEncoder()
scaler = MinMaxScaler()

In [6]:
#vectorising the labels
list_of_coluns_to_encoder = ['school','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob',
'reason','guardian','schoolsup','famsup','paid','activities',
'nursery','higher','internet','romantic', 'Passed', 'sex']
for colum in list_of_coluns_to_encoder:
    data[colum] = encoder.fit_transform(data[colum])

In [7]:
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,Passed
0,0,0,18,1,0,0,4,4,0,4,0,1,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,6,5,6,6,0
1,0,0,17,1,0,1,1,1,0,2,0,0,1,2,0,0,1,0,0,0,1,1,0,5,3,3,1,1,3,4,5,5,6,0
2,0,0,15,1,1,1,1,1,0,2,2,1,1,2,3,1,0,1,0,1,1,1,0,4,3,2,2,3,3,10,7,8,10,1
3,0,0,15,1,0,1,4,2,1,3,1,1,1,3,0,0,1,1,1,1,1,1,1,3,2,2,1,1,5,2,15,14,15,1
4,0,0,16,1,0,1,3,3,2,2,1,0,1,2,0,0,1,1,0,1,1,0,0,4,3,2,1,2,5,4,6,10,10,1


In [8]:
# exploratory work, i will see if some conditions inteferrer to a student pass or not. I will use G3 has parameter to diced
# I will do just for romantic and nursery, because i really don't know if those are statistically significant.
grades_of_students_wich_are_dating = data.query('romantic == 1')['G3']

grades_of_students_wich_are_not_dating = data.query('romantic == 0')['G3']

# the null hypotesses will be: students_wich_are_dating = students_wich_are_not_dating.
# so the anternative hypotesses wil be: mean_deating != mean_not_dating
# if pvalue <= 0.05, it will refuse the null hypotesses, so they will be statistically diferent and i will use for the model.
# if not, i will drop.
r1 = ranksums(grades_of_students_wich_are_dating, grades_of_students_wich_are_not_dating)

grades_of_students_wich_have_nursery = data.query('nursery == 1')['G3']
grades_of_students_wich_not_have_nursery = data.query('nursery == 0')['G3']

r2 = ranksums(grades_of_students_wich_have_nursery,grades_of_students_wich_not_have_nursery)

print(f'pvalue romantic: {r1.pvalue}')
print(f'pvalue nursery: {r2.pvalue}')

pvalue romantic: 0.07063194525553458
pvalue nursery: 0.5302464960777487


In [9]:
data.drop(['G1','G2','G3','nursery','romantic'], axis = 1, inplace = True)

In [10]:
data.count()

school        395
sex           395
age           395
address       395
famsize       395
Pstatus       395
Medu          395
Fedu          395
Mjob          395
Fjob          395
reason        395
guardian      395
traveltime    395
studytime     395
failures      395
schoolsup     395
famsup        395
paid          395
activities    395
higher        395
internet      395
famrel        395
freetime      395
goout         395
Dalc          395
Walc          395
health        395
absences      395
Passed        395
dtype: int64

In [11]:
data.shape

(395, 29)

In [12]:
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,higher,internet,famrel,freetime,goout,Dalc,Walc,health,absences,Passed
0,0,0,18,1,0,0,4,4,0,4,0,1,2,2,0,1,0,0,0,1,0,4,3,4,1,1,3,6,0
1,0,0,17,1,0,1,1,1,0,2,0,0,1,2,0,0,1,0,0,1,1,5,3,3,1,1,3,4,0
2,0,0,15,1,1,1,1,1,0,2,2,1,1,2,3,1,0,1,0,1,1,4,3,2,2,3,3,10,1
3,0,0,15,1,0,1,4,2,1,3,1,1,1,3,0,0,1,1,1,1,1,3,2,2,1,1,5,2,1
4,0,0,16,1,0,1,3,3,2,2,1,0,1,2,0,0,1,1,0,1,0,4,3,2,1,2,5,4,1


In [14]:
x = data.iloc[:, :29].values
x = scaler.fit_transform(x)
y = data.iloc[:,28].values

In [15]:
x.shape

(395, 29)

In [16]:
y = y.reshape(-1,1)
y.shape

(395, 1)

In [17]:
model = KNeighborsClassifier()

In [18]:
params = {
    'n_neighbors' : range(5,40),
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['ball_tree','kd_tree','brute']
}
grid = GridSearchCV(estimator = model, 
param_grid = params,
cv = 10)
grid.fit(x,y)
results = pd.DataFrame(grid.cv_results_)
results_sorted_by_mean_test_score = results.sort_values("mean_test_score", ascending=False)
results_sorted_by_mean_test_score.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
26,0.001496,0.000676,0.003489,0.000918,ball_tree,18,uniform,"{'algorithm': 'ball_tree', 'n_neighbors': 18, ...",1.0,0.975,1.0,1.0,0.925,1.0,0.974359,0.974359,1.0,1.0,0.984872,0.022970,1
166,0.000103,0.000309,0.002479,0.000488,brute,18,uniform,"{'algorithm': 'brute', 'n_neighbors': 18, 'wei...",1.0,0.975,1.0,1.0,0.925,1.0,0.974359,0.974359,1.0,1.0,0.984872,0.022970,1
96,0.001995,0.000441,0.003294,0.000643,kd_tree,18,uniform,"{'algorithm': 'kd_tree', 'n_neighbors': 18, 'w...",1.0,0.975,1.0,1.0,0.925,1.0,0.974359,0.974359,1.0,1.0,0.984872,0.022970,1
162,0.000199,0.000399,0.002093,0.000296,brute,16,uniform,"{'algorithm': 'brute', 'n_neighbors': 16, 'wei...",1.0,0.975,1.0,1.0,0.900,1.0,0.974359,0.974359,1.0,1.0,0.982372,0.029719,4
22,0.001712,0.000477,0.003502,0.000802,ball_tree,16,uniform,"{'algorithm': 'ball_tree', 'n_neighbors': 16, ...",1.0,0.975,1.0,1.0,0.900,1.0,0.974359,0.974359,1.0,1.0,0.982372,0.029719,4


In [29]:
for index, row in results_sorted_by_mean_test_score.head(1).iterrows():
  minimum = row.mean_test_score - (row.std_test_score * 2)
  #maximum = row.mean_test_score + (row.std_test_score * 2) = 1.03
  maximum = 1
print(f'So, the interval of confiance of my model is [%.2f, %.2f]' % (minimum, maximum) )

So, the interval of confiance of my model is [0.94, 1.00]
